In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.
C:\Users\AVANISH SINGHAL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\AVANISH SINGHAL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\AVANISH SINGHAL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [10]:
img_width, img_height = 50, 50

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = './breast-histopathology-images/8867/'
validation_data_dir = './breast-histopathology-images/8914/',
nb_train_samples = 1422
nb_validation_samples = 800
epochs = 10
batch_size = 16

In [11]:
datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    './breast-histopathology-images/8867/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, nb_train_samples / batch_size)
np.save(open('bottleneck_features_trains.npy', 'wb'),
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    './breast-histopathology-images/8914/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, nb_validation_samples / batch_size)
np.save(open('bottleneck_features_validations.npy', 'wb'),
        bottleneck_features_validation)



Found 1422 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [12]:
train_data = np.load(open('bottleneck_features_trains.npy','rb'))
train_labels = np.array(
    [0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

validation_data = np.load(open('bottleneck_features_validations.npy','rb'))
validation_labels = np.array(
    [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=25,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)


Train on 1422 samples, validate on 800 samples
Epoch 1/25
1422/1422 [==============================] - 1s 606us/step - loss: 0.5933 - accuracy: 0.6800 - val_loss: 0.6315 - val_accuracy: 0.6475
Epoch 2/25
1422/1422 [==============================] - 1s 352us/step - loss: 0.5104 - accuracy: 0.7496 - val_loss: 0.6091 - val_accuracy: 0.6525
Epoch 3/25
1422/1422 [==============================] - 0s 347us/step - loss: 0.4840 - accuracy: 0.7665 - val_loss: 0.5984 - val_accuracy: 0.6900
Epoch 4/25
1422/1422 [==============================] - 1s 740us/step - loss: 0.4687 - accuracy: 0.7567 - val_loss: 0.6323 - val_accuracy: 0.6875
Epoch 5/25
1422/1422 [==============================] - 1s 722us/step - loss: 0.4435 - accuracy: 0.7911 - val_loss: 0.6201 - val_accuracy: 0.6938
Epoch 6/25
1422/1422 [==============================] - 1s 724us/step - loss: 0.4436 - accuracy: 0.7876 - val_loss: 0.6166 - val_accuracy: 0.6938
Epoch 7/25
1422/1422 [==============================] - 1s 723us/step - loss:

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 56, 56, 256)      

Epoch 1/10
  1/100 [..............................] - ETA: 55:16 - loss: 0.6906 - accuracy: 0.6250